<a href="https://colab.research.google.com/github/AlinaOtr/data/blob/master/Researching_music_preferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Этап 1. Получение данных

**Постановка задачи:** определить, как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? Сравнить, что и в каком режиме слушают жители Москвы и Петербурга.

Изучу данные, предоставленные сервисом для проекта.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/datasets/music_project.csv')

In [ ]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Всего в таблице 7 столбцов, тип данных у каждого столбца - object.

Подробно разберу, какие в *df* столбцы и какую информацию они содержат:

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.



Итак, каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов (присутствуют заглавные буквы и нужно проверить наличие пробелов). Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.

# Этап 2. Предобработка данных

In [ ]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.
**Переименую столбцы** для удобства дальнейшей работы и проверю результат.


In [ ]:
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday']
df.set_axis(new_names, axis = 'columns', inplace = True)

In [ ]:
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

**Проверю данные на наличие пропусков** вызовом набора методов для суммирования пропущенных значений.

In [ ]:
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем,  не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных.

Заменяю пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'.

In [ ]:
df['track_name'] = df['track_name'].fillna('unknown')


In [ ]:
df['artist_name'] = df['artist_name'].fillna('unknown')

In [ ]:
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Удаляю в столбце с жанрами пустые значения; убеждаюсь, что их больше не осталось.

In [ ]:
df.dropna(subset = ['genre_name'], inplace = True)

In [ ]:
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

Теперь **необходимо установить наличие дубликатов**.  Если найдутся, удалю их, и проверю, все ли удалились.

In [ ]:
df.duplicated().sum()

3755

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)

In [ ]:
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Сохраню список уникальных значений столбца с жанрами в переменной *genres_list*. 

Объявлю функцию *find_genre()* для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.





In [ ]:
genres_list = df['genre_name'].unique()

In [ ]:
def find_genre(genre_name):  
    wrong_genre_name = 0
    for element in genres_list:
        if element == genre_name:
            wrong_genre_name = wrong_genre_name +1
    return wrong_genre_name

Вызову функции *find_genre()* для поиска различных вариантов названия жанра хип-хоп в таблице. Правильное название — *hiphop*. Поищу другие варианты:

* hip
* hop
* hip-hop


In [ ]:
find_genre('hip')

1

In [ ]:
find_genre('hop')

0

In [ ]:
find_genre('hip-hop')

0

Объявлю функцию *find_hip_hop()*, которая заменяет  неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены. Так я исправлю все варианты написания, которые выявила проверка.

In [ ]:
def find_hip_hop(df, wrong):
    df['genre_name'] = df['genre_name'].replace(wrong, 'hiphop')
    count_wrong = df[df['genre_name'] == wrong]['genre_name'].count()
    return count_wrong

In [ ]:
find_hip_hop(df, 'hip')

0

Получаю общую информацию о данных и убеждаюсь, что чистка выполнена успешно.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
user_id        60126 non-null object
track_name     60126 non-null object
artist_name    60126 non-null object
genre_name     60126 non-null object
city           60126 non-null object
time           60126 non-null object
weekday        60126 non-null object
dtypes: object(7)
memory usage: 3.2+ MB


**Вывод:**

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

# Этап 3. Анализ

## Действительно ли музыку в разных городах слушают по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для каждого города устанавлю количество прослушанных  в эти дни композиций с известным жанром и сравнивню результаты.

In [ ]:
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

In [ ]:
df.groupby('weekday')['genre_name'].count()


weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаю функцию *number_tracks()*, которая возвращает количество прослушанных композиций, для которых известен жанр,  в заданный день недели в заданном городе

In [ ]:
def number_tracks(df, day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [ ]:
number_tracks(df, 'Monday', 'Moscow')

15347

In [ ]:
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [ ]:
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [ ]:
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [ ]:
number_tracks(df, 'Friday', 'Moscow')

15680

In [ ]:
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Сведу полученную информацию в одну таблицу


In [ ]:
columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow', 15347, 10865, 15680], ['Saint-Petersburg', 5519, 6913, 5802]]
table = pd.DataFrame(data = data, columns = columns)
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


**Вывод:**

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

## Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Теперь найду ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

In [ ]:
moscow_general = df[df['city'] == 'Moscow']

In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создам функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [ ]:
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    genre_list_sorted = genre_list.groupby('genre_name')['genre_name'].count().sort_values(ascending = False).head(10)
    return genre_list_sorted

Cравню полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Вывод:**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

## Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

**Гипотеза:** Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.



In [ ]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [ ]:
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [ ]:
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [ ]:
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Вывод:**

В Москве кроме абсолютно популярного жанра поп есть направление русской популярной музыки. Это значит, что интерес к данному жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

# Этап 4. Результаты исследования


Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

**Общие результаты:**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней недели наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза не подтверждена и третья не подтверждена.